Generate Brochure

In [41]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI


In [44]:
load_dotenv(override=True)
api_key=os.getenv("OPENAI_API_KEY")
MODEL='gpt-4o-mini'
openai=OpenAI()

In [ ]:
#Class that scrap the link from webpages
# Some websites need you to use proper headers when fetching them:

headers={
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self,url):
        self.url=url
        response=requests.get(url,headers=headers)
        self.body=response.content
        soup=BeautifulSoup(self.body, 'html.parser')
        self.title=soup.title.string if soup.title else "No title found"

        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()

            self.text=soup.body.get_text(separator="\n",strip=True)
        else:
            self.text = ""

        links=[link.get("href") for link in soup.find_all('a')]
        self.links=[link for link in links if link]
    
    def get_content(self):
        return f"Webpage Title: \n{self.title}\nWebpage Contents:\n{self.text}\n\n"
    


In [80]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

OpenAI to create brochure for Time of India

In [81]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [82]:
def get_links(url):
    website=Website(url)
    response=openai.chat.completions.create(model=MODEL,messages=[
        {"role":"system","content": link_system_prompt},
        {"role":"user","content":get_links_user_prompt(website)}
    ],
    response_format={"type":"json_object"}
    )
    result=response.choices[0].message.content
    return json.loads(result)

In [83]:
def get_all_details(url):
    result="landing page:\n"
    result+= Website(url).get_content()
    links=get_links(url)
    print("Found links:" ,links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result+=Website(link["url"]).get_content()
    return result

In [88]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information. instead of making it in English create it in Hindi language"

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information. instead of making it in English create it in Hindi language."


In [89]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You're looking at a news publishing company called: {company_name}\n"

    user_prompt+=f"Here are the content of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"

    user_prompt+= get_all_details(url)

    user_prompt= user_prompt[:5_0000] # truncate if more than 5000 character

    return user_prompt

In [90]:
def create_brochure(company_name, url):
    response=openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role":"system","content":system_prompt},
            {"role":"user","content":get_brochure_user_prompt(company_name,url)}
        ]
    )
    result=response.choices[0].message.content
    display(Markdown(result))

In [91]:
create_brochure("Times of India","https://timesofindia.indiatimes.com/")

Found links: {'links': [{'type': 'homepage', 'url': 'https://timesofindia.indiatimes.com/'}, {'type': 'about page', 'url': 'https://www.timesinternet.in'}, {'type': 'careers page', 'url': 'https://timesinternet.in/careers'}]}


```markdown
# टाइम्स ऑफ इंडिया - मीडिया का मसाला!

### क्या है टाइम्स ऑफ़ इंडिया?
टाइम्स ऑफ इंडिया, यानि TOI, हमें डेटा और हेडलाइन्स का ऐसा कॉम्बो देता है, जैसे चाय में दो चम्मच चीनी! यहाँ आपको मिलते हैं सबसे ताज़ा ख़बरें, खेल, बिज़नेस की स्टोरीज, और बॉलीवुड की गपशप।

### हमारी संस्कृति - "काम करो, मस्त रहो"
हमारी ऑफिस में काम करने की संस्कृति बस यही है: "परफॉर्म करो, मस्ती करो!" साप्ताहिक रिपोर्टिंग मीटिंग्स में ठीक तरह से कुछ 'क्रिकेट' की गपशप जरूरी समझी जाती है। एक साथ मिलकर हंसी-मजाक करने का मजा हम कभी नहीं छोड़ते।

### हमारे ग्राहक - आप सब
हमारे ग्राहक? बस सभी! हर उम्र और किस्म के लोग - अधिकारियों से लेकर कॉलेज के छात्रों तक, टाइम्स ऑफ इंडिया में सबको कुछ ना कुछ मिल जाता है। 

### करियर का मौका - चलो, साथ में बनाते हैं स्टोरीज!
क्या आप रिपोर्टर बनना चाहते हैं, या फिर खेल संवाददाता? हमारे पास जॉब्स हैं जो आपकी प्रतिभा को चमकाने के लिए बिलकुल सही हैं। यहाँ हर दिन नए मौके हैं, और हम आपको खुद को साबित करने का मौका देते हैं। 

### तो क्यों ना जुड़ें!
क्या आप कुछ नया और मस्त करना चाहते हैं? तब पर आज ही टाइम्स ऑफ इंडिया की टीम में शामिल हों। यहाँ मौका है न केवल अच्छी ख़बरें लिखने का, बल्कि एक मजेदार टीम का हिस्सा बनने का भी।

### अंत में, हम हैं "टाइम्स ऑफ इंडिया" - एक मजेदार मीडिया सफर का हिस्सा बनिए!
जब भी कुछ बड़ा हो – आपको पता लगाने के लिए हमें फॉलो करें। हम रहते हैं हर सिटि में, इंग्लिश में नहीं, आपके अपनी ज़ुबान में भी!

**तुम्हारा आज का समय, तुम्हारी खुशियों का साया। क्या आप तैयार हैं?**
```
